In [156]:
import matplotlib.pyplot as plt
from utils.util_metrics import *
from utils.util_draw import *
import warnings
warnings.filterwarnings('ignore')
import utils.ai_plotting as aiplt
import matplotlib.pyplot as plt
import json
import glob
import natsort as ns
import seaborn as sns
print(plt.style.available)
aiplt.aingura_plot_style(grid_color='white', plt_style='seaborn-v0_8-white')


['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


#### Read Results

In [161]:
exp = f'results/exp_real/Mfix/tryreals_no_pa/'
paths = ns.natsorted(glob.glob(f'{exp}/simu*.json'))
print(paths)
# Load results from JSON files

all_exps = []
all_exps_std = []
# models = ['1.SPBN','2.BSBN', '4.BSBN-FKDE', '5.GBN-BIC', '6.GBN-BGe']
models = ['1.SPBN','2.BSBN', '5.GBN-BIC', '6.GBN-BGe']
for path in paths:
    M = int(path.split('.')[0].split('_')[-1])
    with open(path, 'r') as json_file:
        results = json.load(json_file)
    
        dataframes = []
        model_index, instances_col,simucol = [],[],[]
        for nm, model in enumerate(models):
            instances = ns.natsorted(results[model].keys())
            for n in instances:
                simus = results[model][n].keys()
                for simu in simus:
                    ## PREPARE DATAFRAMES
                    reader = ExperimentsReader(results[model][n][simu], model, 'REF')
                    df_all = reader.return_dataframe(simulated=True).reset_index().drop(columns=['model'], axis=1)
                    
                    reader0 = ExperimentsReader(results[models[0]][n][simu], models[0], 'REF')
                    df0 = reader0.return_dataframe(simulated=True).reset_index()

                    df_all['HC Ratio'] = df0['train']/df_all['train']
                    df_all['Test Ratio'] = df0['test']/df_all['test']
                    df_all['Logl diff'] = df_all['slogl_REF'] - df_all['slogl']
                    

                    df_mean = df_all.mean().to_frame().T
                    df_std = df_all.std().to_frame().T
                    dataframes.append((df_all, df_mean, df_std))
            

                    model_index.append(model[2:])                 
                    instances_col.append(n) 
                    simucol.append(int(simu[-1]))

        all_df= pd.concat([df_mean for _, df_mean,_ in dataframes]).reset_index(drop=True).drop(columns=['index'], axis=1)
        all_df.index = model_index
        all_df.index.name = 'Model'
        all_df['Instances'] = instances_col
        all_df['Dataset'] = simucol
        all_df['M'] = [str(M)]*len(all_df)


        all_df_std= pd.concat([df_std for _, _,df_std in dataframes]).reset_index(drop=True).drop(columns=['index'], axis=1)
        all_df_std.index = model_index
        all_df_std.index.name = 'Model'
        all_df_std['Instances'] = instances_col
        all_df_std['Dataset'] = simucol
        all_df_std['M'] = [str(M)]*len(all_df_std)


        all_exps.append(all_df)
        all_exps_std.append(all_df_std)
        
concat_all = pd.concat(all_exps).reset_index()
concat_all_std = pd.concat(all_exps_std).reset_index()

['results/exp_real/Mfix/tryreals_no_pa/simu_all_50.json']


In [162]:
concat_all

,Model,hamming,shamming,thamming,rmse,rmae,train,test,test_REF,slogl,slogl_REF,HC Ratio,Test Ratio,Logl diff,Instances,Dataset,M
0,SPBN,0.0,0.0,0.0,0.000000,0.000000,1981.555984,7.697741,7.697741,-61937.957891,-61937.957891,1.000000,1.000000,0.000000,16384,0,50
1,SPBN,0.0,0.0,0.0,0.000000,0.000000,356.044323,3.140523,3.140523,-41128.409478,-41128.409478,1.000000,1.000000,0.000000,16384,1,50
2,SPBN,0.0,0.0,0.0,0.000000,0.000000,255.571753,2.784677,2.784677,-11016.907262,-11016.907262,1.000000,1.000000,0.000000,16384,2,50
3,SPBN,0.0,0.0,0.0,0.000000,0.000000,769.834558,5.016494,5.016494,-54448.506147,-54448.506147,1.000000,1.000000,0.000000,16384,3,50
4,BSBN,1.2,12.0,0.0,3.939129,2.220073,1667.403288,4.473784,7.697741,-63220.873884,-61937.957891,1.193203,1.722485,1282.915993,16384,0,50
5,BSBN,1.2,1.2,0.0,0.488459,1.907080,185.473262,0.327865,3.140523,-41767.582965,-41128.409478,1.923353,9.645409,639.173488,16384,1,50
6,BSBN,3.2,4.2,1.0,2.368476,12.134878,123.153358,0.199847,2.784677,-12305.887949,-11016.907262,2.076113,14.061783,1288.980687,16384,2,50
7,BSBN,2.8,5.6,0.0,0.742934,1.444987,560.846744,2.498336,5.016494,-54713.223616,-54448.506147,1.388892,2.334171,264.717469,16384,3,50
8,GBN-BIC,4.8,21.0,9.0,24.309555,16.664300,0.140592,0.004106,7.697741,-73595.143063,-61937.957891,14383.135586,1915.831712,11657.185171,16384,0,50
9,GBN-BIC,11.6,19.6,8.0,8.638005,21.676717,0.060504,0.004550,3.140523,-53681.648230,-41128.409478,5894.089982,705.498181,12553.238753,16384,1,50


#### Prepare Table and CDD

In [163]:
results = concat_all[['Dataset','Model', 'M', 'Instances', 'slogl', 'HC Ratio', 'Test Ratio']].sort_values(by=['Dataset','Model']).reset_index(drop=True).round(2)
results_std = concat_all_std[['Dataset','Model', 'M','Instances', 'slogl', 'HC Ratio', 'Test Ratio']].sort_values(by=['Dataset','Model']).reset_index(drop=True).round(2)

results_with_std = results.copy()
results_with_std['HC Ratio'] =  results['HC Ratio'].astype(str) + ' $\pm$ ' + results_std['HC Ratio'].astype(str)
results_with_std['Test Ratio'] =  results['Test Ratio'].apply(lambda x: f"{x:.2e}").astype(str) + ' $\pm$ ' + results_std['Test Ratio'].astype(str) 
results_with_std['Dataset'] = results_with_std['Dataset'] + 1

##### Ratio Table

In [164]:
results_with_std_bsbns = results_with_std[results_with_std['Model'].str.contains('BSBN')].reset_index(drop=True)
# results_with_std_bsbns[['Dataset','Model', 'M', 'HC Ratio', 'Test Ratio']].to_csv(f'{exp}/results_1PA.csv', index=False)
results_with_std_bsbns

,Dataset,Model,M,Instances,slogl,HC Ratio,Test Ratio
0,1,BSBN,50,16384,-63220.87,1.19 $\pm$ 0.28,1.72e+00 $\pm$ 0.16
1,2,BSBN,50,16384,-41767.58,1.92 $\pm$ 0.23,9.65e+00 $\pm$ 1.02
2,3,BSBN,50,16384,-12305.89,2.08 $\pm$ 0.12,1.41e+01 $\pm$ 2.14
3,4,BSBN,50,16384,-54713.22,1.39 $\pm$ 0.18,2.33e+00 $\pm$ 1.34


##### CDD Format
The bergamn-hommel analysis is performed in Rstudio. Then the CDD is plotted in python again

In [155]:
prepare_CD = results_with_std[['Model', 'Dataset','M', 'Instances', 'slogl']]
prepare_CD['data_grid_instances'] = prepare_CD['Dataset'].astype(str) + '_' + prepare_CD['M'] + '_' + prepare_CD['Instances']
slogl_cd = prepare_CD[['Model', 'data_grid_instances', 'slogl']]
slogl_cd.columns = ['network', 'dataset_name', 'metric']
transformed_slogl_cd = slogl_cd.pivot(index='dataset_name', columns='network', values='metric')

path = f"critical_diff/real_1pa"
os.makedirs(path, exist_ok=True)

transformed_slogl_cd.to_csv(path+ "/slogl.csv", index=False)

network,BSBN,BSBN-FKDE,GBN-BGe,GBN-BIC,SPBN
dataset_name,,,,,
1_100_16384,-72910.62,-70755.71,-75429.51,-75429.51,-72821.55
1_50_16384,-73412.19,-71809.63,-75429.51,-75429.51,-72877.69
2_100_16384,-41849.08,-42388.74,-55526.09,-55657.12,-41722.29
2_50_16384,-42248.63,-45995.64,-55526.09,-55657.12,-41709.66
3_100_16384,-12764.97,-9365.02,-25477.02,-25477.02,-12042.77
3_50_16384,-12946.84,-13583.68,-25477.02,-25477.02,-12042.77
4_100_16384,-55554.14,-39750.10,-64900.05,-64851.27,-55487.86
4_50_16384,-55741.17,-53083.70,-64900.05,-64851.27,-55487.86
5_100_16384,-101468.54,-100855.07,-111715.45,-111649.13,-101230.27
